# Create and run a ZONEBUDGET on a general MODFLOW model

This notebook implements a grid-search approach to finding hydraulic conductivities that result in heads that seem reasonable.

Project specific variables are imported in the model_spec.py and gen_mod_dict.py files that must be included in the notebook directory. The first first includes pathnames to data sources that will be different for each user. The second file includes a dictionary of model-specific information such as  cell size, default hydraulic parameter values, and scenario defintion (e.g. include bedrock, number of layers, etc.). There are examples in the repository. Run the following cells up to the "Run to here" cell to get a pull-down menu of models in the model_dict. Then, without re-running that cell, run all the remaining cells.  Re-running the following cell would re-set the model to the first one in the list, which you probably don't want. If you use the notebook option to run all cells below, it runs the cell you're in, so if you use that option, move to the next cell (below the pull-down menu of models) first.

In [1]:
__author__ = 'Jeff Starn'
%matplotlib notebook
from model_specs import *
from gen_mod_dict import *

import os, sys
import shutil
import numpy as np
import matplotlib.pyplot as plt

from flopy.utils.postprocessing import get_water_table


import flopy.utils.binaryfile as bf
from matplotlib import colors
import flopy as fp
import pandas as pd
# import ipyparallel as ipp
# from model_specs import *
# from gen_mod_dict import *

from ipywidgets import interact, Dropdown
from IPython.display import display

The next cell is a template for making this notebook into a batch script. To do so, save this notebook as a .py file and edit it as follows. Comment out all the notebook-specific commands (drop-down menu stuff and commands preceded by %). Indent everything below the next cell twice so that it falls within the 'for' loop and the 'try' statement. Move the 'except' statement to the end of the script. Comment out lines in the cell after 'Preliminary stuff' so that the model is selected in the 'for' loop from gen_mod_dict. You can leave the print statement in that cell uncommented. 

In [2]:
for key, value in model_dict.items():   # from "gen_mod_dict.py"
    md = key
    ms = model_dict[md]
    print('trying {}'.format(md))
    try:
        pass
    except:
        pass

trying Assabet
trying CoastalCT


In [3]:
models = list(model_dict.keys())
models.sort()
model_area = Dropdown(
    options=models,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

### Run to here to initiate notebook

First time using this notebook in this session (before restarting the notebook), run the cells up to this point. Then select your model from the dropdown list above. Move your cursor to this cell and use the toolbar menu Cell --> Run All Below.  After the first time, if you want to run another model, select your model and start running from this cell--you don't need to re-run the cells from the beginning.

## Preliminary stuff

In [4]:
md = model_area.value
ms = model_dict[md]
print('The model being processed is {}\n'.format(md))

The model being processed is CoastalCT



In [5]:
hydro_wt = 1.0
calib=False
PEST=True
label = "tillBedrock_0_round_params_RIV"

geo_ws = os.path.join(proj_dir, ms['ws'])
parent_ws = os.path.join(geo_ws, scenario_dir)
if calib:
    dir_name = '{}_cal_wt_{:4.2f}'.format(scenario_dir, hydro_wt)
else:
    dir_name = scenario_dir
    
if PEST:
    dir_name = dir_name + "_PEST"
    
if label!='':
    dir_name = dir_name +"_"+str(label)
    
model_ws = os.path.join(geo_ws, dir_name)

if not os.path.exists(model_ws):
    print('This calibration scenario has not been completed')

In [6]:
model_file = os.path.join(geo_ws, 'model_grid.csv')
model_grid = pd.read_csv(model_file, index_col='node_num', na_values=['nan', hnoflo])

NROW = model_grid.row.max() + 1
NCOL = model_grid.col.max() + 1
num_cells = NROW * NCOL
NLAY = num_surf_layers + 1

zones = model_grid.zone.values.reshape(NROW, NCOL)
zones = zones.astype(int)
zone_array = np.repeat(zones[np.newaxis,:,:],NLAY,axis=0)

#change 0 to 1000
zone_array[zone_array==0]=1000


In [7]:
cbc_file = os.path.join(model_ws,'{}.cbc'.format(md))
zb = fp.utils.zonbud.ZoneBudget(cbc_file,zone_array)

C:\Users\jbarclay\.conda\envs\genmod\lib\site-packages\flopy\utils\zonbud.py:1424: RuntimeWarning: invalid value encountered in double_scalars
  flux = 100 * in_minus_out / (in_plus_out / 2.)


In [8]:
#extract the zonebudget results and change it to 1 row per zone
names = zb.get_record_names()
names = names.tolist()
nameLst = [x for x in names if "ZONE" not in x]
zb_df = zb.get_dataframes(names=nameLst)
zb_df.reset_index(level="totim", drop=True, inplace=True)
zb_df = zb_df.transpose()
zb_df['Zone_Nm']=zb_df.index
zb_df.reset_index(drop=True,inplace=True)

In [9]:
#add a zone column and populate it with the zone number
zb_df['Zone']=[int(x) for x in (zb_df.Zone_Nm.str[5:])]
#change zone 0 back to 0
#zb_df['Zone']==1000
zb_df.loc[zb_df.Zone==1000,'Zone']=0

#calculate some new columns
zb_df['TRANSFERS_IN']=zb_df['TOTAL_IN']-(zb_df[['CONSTANT_HEAD_IN', 'DRAINS_IN','HEAD_DEP_BOUNDS_IN', 'RECHARGE_IN',]].sum(axis=1))
zb_df['TRANSFERS_OUT']=zb_df['TOTAL_OUT']-(zb_df[['CONSTANT_HEAD_OUT', 'DRAINS_OUT','HEAD_DEP_BOUNDS_OUT', 'RECHARGE_OUT',]].sum(axis=1))
zb_df['CGWD_FRAC']=zb_df['HEAD_DEP_BOUNDS_OUT']/zb_df['TOTAL_OUT']
zb_df['DRN_FRAC']=zb_df['DRAINS_OUT']/zb_df['TOTAL_OUT']
zb_df['TRANS_FRAC']= zb_df['TRANSFERS_OUT']/zb_df['TOTAL_OUT']
zb_df['CGWD_DRN_ratio']=zb_df['HEAD_DEP_BOUNDS_OUT']/zb_df['DRAINS_OUT']

#change inf values to nan
zb_df.loc[np.isinf(zb_df.CGWD_DRN_ratio),'CGWD_DRN_ratio']=np.nan

In [10]:
f_out = os.path.join(model_ws, 'Zone_Output.csv')
zb_df.to_csv(f_out)

In [11]:
zb_df[zb_df.Zone==18]

name,CONSTANT_HEAD_IN,CONSTANT_HEAD_OUT,DRAINS_IN,DRAINS_OUT,HEAD_DEP_BOUNDS_IN,HEAD_DEP_BOUNDS_OUT,RECHARGE_IN,RECHARGE_OUT,RIVER_LEAKAGE_IN,RIVER_LEAKAGE_OUT,...,WELLS_IN,WELLS_OUT,Zone_Nm,Zone,TRANSFERS_IN,TRANSFERS_OUT,CGWD_FRAC,DRN_FRAC,TRANS_FRAC,CGWD_DRN_ratio
17,0.0,0.0,0.0,360004.75,542880.8125,609905.0,436006.0,0.0,88516.6875,544.825012,...,0.0,3610.489502,ZONE_18,18,115473.6875,124450.75,0.557316,0.328964,0.11372,1.694158
